In [1]:
data_var = '2023-12-29'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10105,2023-12-29,Japão B.League,06:05,Akita,Nagoya Fighting Eagles,1.55,2.34,146.5,1.87,1.79,-4.5,1.99,1.73,b7YUqBeP,0.645161,0.427350,0.534759,0.558659,0.072512,110.390,16.686947,0.151164,2.4,1.341641,0.559017,113.52,1.408,0.123167,0.087476,71.0,149.232,37.530993,0.251494,1.2,1.643168,1.369306,200.76,1.954,0.418187,0.214016,-8.0,88,84,1.29,2.39,281.184,176.768,0.287205,0.030912,0.098843,-0.24,-0.048,-11.458333,0.000000,0.0,0.000000,2.13,0.426,3.145540,0.000000,0.0,0.000000
10106,2023-12-29,Japão B.League,06:05,Saga,Alvark,3.44,1.28,145.5,1.83,1.87,7.5,1.96,1.75,zTPduV3t,0.290698,0.781250,0.546448,0.534759,0.071948,224.382,83.170726,0.370666,1.8,1.643168,0.912871,150.96,2.590,1.012694,0.391002,34.0,95.846,18.720102,0.195314,2.4,1.341641,0.559017,71.39,1.234,0.093434,0.075717,38.0,102,59,1.48,1.21,142.010,96.928,0.647182,0.015289,0.080050,3.41,0.682,3.577713,0.000000,0.0,0.000000,0.76,0.152,1.842105,0.000000,0.0,0.000000
10107,2023-12-29,Japão B.League,07:05,Brave Thunders,Diamond Dolphins,1.88,1.83,157.5,1.86,1.84,-1.5,2.07,1.68,UkP0vkJn,0.531915,0.546448,0.537634,0.543478,0.078363,98.752,24.555883,0.248662,0.6,1.341641,2.236068,102.51,1.326,0.233944,0.176429,-34.0,180.304,108.861220,0.603765,1.2,1.643168,1.369306,334.56,2.350,1.443260,0.614153,16.0,67,82,1.53,4.08,85.050,186.154,0.019059,0.007644,0.147078,-2.76,-0.552,-1.594203,0.000000,0.0,0.000000,-0.92,-0.184,-4.510870,0.000000,0.0,0.000000
10108,2023-12-29,Japão B2.League,02:00,Shiga,Shizuoka,1.22,3.98,156.5,1.80,1.90,-10.5,2.00,1.72,juAlqGpU,0.819672,0.251256,0.555556,0.526316,0.070928,119.260,37.756958,0.316594,2.4,1.341641,0.559017,100.44,1.324,0.174728,0.131970,31.0,284.398,182.618765,0.642124,1.2,1.643168,1.369306,210.32,3.568,2.272272,0.636848,-24.0,81,88,1.24,2.39,111.014,285.532,0.750621,0.038222,0.106446,1.14,0.228,0.964912,0.000000,0.0,0.000000,-3.56,-0.712,-4.185393,0.000000,0.0,0.000000
10109,2023-12-29,Japão B2.League,05:00,Iwate,Ehime Orange Vikings,1.71,2.03,150.5,1.81,1.86,-2.5,1.97,1.74,S01Gvzgo,0.584795,0.492611,0.552486,0.537634,0.077406,336.582,183.657508,0.545655,0.0,0.000000,NaN,581.40,5.132,2.811027,0.547745,-39.0,311.216,249.883279,0.802926,1.2,1.643168,1.369306,111.22,3.540,2.277422,0.643339,-6.0,68,83,8.55,1.34,184.186,0.000,0.121002,0.019267,0.087674,-5.00,-1.000,-0.710000,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10249,2023-12-29,China Cba,08:35,Zhejiang Chouzhou,Beijing,1.31,3.32,193.5,1.85,1.89,-9.5,2.03,1.68,b7rtDwmL,0.763359,0.301205,0.540541,0.529101,0.064564,141.780,27.603862,0.194695,2.4,1.341641,0.559017,167.99,1.310,0.190788,0.145640,37.0,138.822,15.872748,0.114339,1.2,1.643168,1.369306,122.08,1.424,0.283954,0.199406,-18.0,107,109,1.57,1.12,120.322,175.212,0.613946,0.015125,0.133416,0.86,0.172,1.802326,0.852651,1.0,0.147349,-1.69,-0.338,-6.863905,0.673133,0.4,-0.273133
10250,2023-12-29,China Wcba Feminina,08:30,Shaanxi Tianze F,Hebei F,2.93,1.37,147.5,1.85,1.85,4.5,2.04,1.66,rHRIBzvg,0.341297,0.729927,0.540541,0.540541,0.071224,369.652,282.027411,0.762954,1.2,1.643

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
10107,07:05,Japão B.League,Brave Thunders,Diamond Dolphins,1.88,Back Home
10109,05:00,Japão B2.League,Iwate,Ehime Orange Vikings,1.71,Back Home
10113,14:00,Lituânia Nkl,Silute,Jurbarkas,2.20,Back Home
10118,03:05,Japão B.League,Ibaraki Robots,Kyoto,2.26,Back Home
10121,14:00,Europa Liga Letônia,BK Ogre,Rigas Zelli,2.03,Back Home
10122,16:45,Itália Série A2,JuVi Cremona,Sebastiani Rieti,1.62,Back Home
10129,13:00,Europa Liga Aba,Krka,FMP,1.64,Back Home
10135,07:00,Israel Liga Leumit,Safed,Ramat Hasharon,1.74,Back Home
10136,07:30,Israel Liga Leumit,Elitzur Shomron,Maccabi Rehovot,1.99,Back Home
10138,13:00,Itália Série A1 Feminina,Battipaglia F,Brixia F,2.31,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO UNDER V1 (QCUT) ------------



,Time,League,Home,Away,Odds_Under,Bet
10138,13:00,Itália Série A1 Feminina,Battipaglia F,Brixia F,1.85,Back Home
10161,16:45,Reino Unido Bbl,Caledonia Gladiators,London Lions,1.85,Back Home


### Back Home V3 (QCut)

In [11]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
10105,06:05,Japão B.League,Akita,Nagoya Fighting Eagles,1.55,Back Home
10107,07:05,Japão B.League,Brave Thunders,Diamond Dolphins,1.88,Back Home
10121,14:00,Europa Liga Letônia,BK Ogre,Rigas Zelli,2.03,Back Home
10149,22:00,Eua Ncaa,Nebraska O.,Denver,1.91,Back Home
10154,22:00,Eua Ncaa,UMKC,Oral Roberts,2.61,Back Home
10208,16:30,Espanha Acb,Manresa,Joventut,1.70,Back Home
10213,21:00,Eua Nba,Orlando Magic,New York Knicks,1.89,Back Home
10222,00:00,Eua Ncaa,CS Fullerton,Long Beach State,2.28,Back Home
10226,14:00,Eua Ncaa,IUPUI,Detroit,2.00,Back Home
10245,08:35,China Cba,Qingdao,Guangzhou,1.54,Back Home
